<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Diego Espinoza
- Nombre de alumno 2: Juan Miño


### **Link de repositorio de GitHub:** [Laboratorio 6: MDS7202](https://github.com/juansebm/MDS7202)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv("./sales.csv")

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [20]:
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
import pickle
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
set_config(transform_output="pandas")

df["date"] = pd.to_datetime(df["date"], dayfirst=True)

X = df.drop(columns="quantity")
y = df["quantity"]
RANDOM_STATE = 1300

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.10, random_state=RANDOM_STATE)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=2/9, random_state=RANDOM_STATE)

In [12]:
def extract_date_parts(df):
    df = df.copy()
    df["day"] = df["date"].dt.day.astype("category")
    df["month"] = df["date"].dt.month.astype("category")
    df["year"] = df["date"].dt.year.astype("category")
    return df.drop(columns=["date"])

date_transformer = FunctionTransformer(extract_date_parts)

In [ ]:
# Preprocesar una muestra y detectar columnas
X_sample = extract_date_parts(X_train)
categoricas = X_sample.select_dtypes(include="category").columns.tolist()
numericas = X_sample.select_dtypes(include=np.number).columns.tolist()

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categoricas),
    ("num", "passthrough", numericas)
])

preprocessor.set_output(transform="pandas")

ColumnTransformer(transformers=[('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['day', 'month', 'year']),
                                ('num', 'passthrough',
                                 ['id', 'lat', 'long', 'pop', 'price'])])

In [18]:
pipe_dummy = Pipeline([
    ("date_parts", date_transformer),
    ("preproc", preprocessor),
    ("model", DummyRegressor(strategy="mean"))
])

In [21]:
pipe_dummy.fit(X_train, y_train)
y_val_pred_dummy = pipe_dummy.predict(X_val)

mae_dummy = mean_absolute_error(y_val, y_val_pred_dummy)
print(f"MAE DummyRegressor: {mae_dummy:.2f}")

MAE DummyRegressor: 13595.80


In [ ]:
pipe_xgb = Pipeline([
    ("date_parts", date_transformer),
    ("preproc", preprocessor),
    ("model", XGBRegressor(random_state=RANDOM_STATE))
])

pipe_xgb.fit(X_train, y_train)
y_val_pred_xgb = pipe_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)
print(f"MAE XGBRegressor: {mae_xgb:.2f}")

MAE XGBRegressor: 7090.66


In [23]:
with open("dummy_model.pkl", "wb") as f:
    pickle.dump(pipe_dummy, f)

with open("xgb_model.pkl", "wb") as f:
    pickle.dump(pipe_xgb, f)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [ ]:
# Ahora textuales
categoricas = ["container", "brand"]
numericas = ["price", "pop"]

preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categoricas),
    ("num", "passthrough", numericas)
])

preprocessor.set_output(transform="pandas")

ColumnTransformer(transformers=[('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['container', 'brand']),
                                ('num', 'passthrough', ['price', 'pop'])])

In [34]:
X_train_transformed = preprocessor.fit_transform(X_train)
feature_names = X_train_transformed.columns.tolist()

In [37]:
monotone_constraints = [0] * len(feature_names)
price_index = feature_names.index("num__price")
monotone_constraints[price_index] = -1

In [ ]:
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

pipe_xgb = Pipeline([
    ("preproc", preprocessor),
    ("xgb", XGBRegressor(monotone_constraints={"num__price": -1}, random_state=RANDOM_STATE)) # RANDOM definido + arriba
])

pipe_xgb.fit(X_train, y_train)

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['container', 'brand']),
                                                 ('num', 'passthrough',
                                                  ['price', 'pop'])])),
                ('xgb',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, device=None,
                              early_stopping_rou...
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints={'num__price': -1},
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=1300, ...))])

In [ ]:
from sklearn.metrics import mean_absolute_error

y_val_pred = pipe_xgb.predict(X_val)W
mae_xgb = mean_absolute_error(y_val, y_val_pred)
print(f"MAE XGBRegressor con restricción monótona: {mae_xgb:.2f}")

MAE XGBRegressor con restricción monótona: 7454.89


*Aunque el MAE aumentó un poco, el modelo con restricción sigue siendo competitivo y más interpretable, por lo que es útil en aplicaciones reales.¨*

In [42]:
import pickle

with open("xgb_model_monotonic.pkl", "wb") as f:
    pickle.dump(pipe_xgb, f)

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
# pip install optuna

  Using cached optuna-4.3.0-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.3.0-py3-none-any.whl (386 kB)
Using cached alembic-1.15.2-py3-none-any.whl (231 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached mako-1.3.10-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import optuna
from optuna.samplers import TPESampler

# FunctionTransformer para fecha
def extract_date_parts(df):
    df = df.copy()
    df["day"] = df["date"].dt.day.astype("category")
    df["month"] = df["date"].dt.month.astype("category")
    df["year"] = df["date"].dt.year.astype("category")
    return df.drop(columns=["date"])

date_transformer = FunctionTransformer(extract_date_parts)

categoricas = ["container", "brand"]
numericas = ["price", "pop"]

# Función objetivo ---> minimizar MAE
def objective(trial):
    # --------- Hiperparámetros a optimizar !!!
    learning_rate    = trial.suggest_float("learning_rate", 0.001, 0.1)
    n_estimators     = trial.suggest_int("n_estimators", 50, 1000)
    max_depth        = trial.suggest_int("max_depth", 3, 10)
    max_leaves       = trial.suggest_int("max_leaves", 0, 100)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    reg_alpha        = trial.suggest_float("reg_alpha", 0.0, 1.0)
    reg_lambda       = trial.suggest_float("reg_lambda", 0.0, 1.0)
    min_freq         = trial.suggest_float("min_frequency", 0.0, 1.0)

    # --------- Preprocesador con min_frequency ---------
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False, min_frequency=min_freq)
    preprocessor = ColumnTransformer([
        ("cat", ohe, categoricas),
        ("num", "passthrough", numericas)
    ])
    preprocessor.set_output(transform="pandas")

    # --------- Modelo con restricciones monótonas ---------
    xgb_model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        monotone_constraints={"num__price": -1},
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    # --------- Pipeline definición ---------
    pipeline = Pipeline([
        ("date_parts", date_transformer),
        ("preproc", preprocessor),
        ("xgb", xgb_model)
    ])

    # --------- Training y evaluación ---------
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_pred)

    # Guardar el besto pipeline
    trial.set_user_attr("best_pipeline", pipeline)

    return mae


In [ ]:
# 1️⃣ Crear el sampler TPE y el optuna estudio
sampler = TPESampler(seed=RANDOM_STATE)

study = optuna.create_study(
    direction="minimize",
    sampler=sampler
)

# Optimizar x 5 minutos
study.optimize(objective, timeout=60 * 5)  # 5 minutos :p

# Reporte 
print(f"✅ N° de trials: {len(study.trials)}")
print(f"✅ Mejor MAE: {study.best_value:.2f}")
print("✅ Mejores hiperparámetros:")
for k, v in study.best_params.items():
    print(f"   - {k}: {v}")

# Guardar el besto pipeline
best_pipeline = study.best_trial.user_attrs["best_pipeline"]

import pickle
with open("xgb_model_optuna.pkl", "wb") as f:
    pickle.dump(best_pipeline, f)


[I 2025-05-01 21:25:35,068] A new study created in memory with name: no-name-91d66ac7-da78-4fc4-8ffb-81dd617588e6
[I 2025-05-01 21:26:09,368] Trial 0 finished with value: 9300.885801749959 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352}. Best is trial 0 with value: 9300.885801749959.
[I 2025-05-01 21:26:19,205] Trial 1 finished with value: 7129.372615865344 and parameters: {'learning_rate': 0.06051038616257767, 'n_estimators': 723, 'max_depth': 3, 'max_leaves': 97, 'min_child_weight': 5, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.18182496720710062, 'min_frequency': 0.18340450985343382}. Best is trial 1 with value: 7129.372615865344.
[I 2025-05-01 21:26:32,792] Trial 2 finished with value: 9284.100148709786 and parameters: {'learning_rate': 0.03111998205299424, 'n_estimators': 549, 'max_depth

✅ N° de trials: 25
✅ Mejor MAE: 7111.45
✅ Mejores hiperparámetros:
   - learning_rate: 0.07728066836343214
   - n_estimators: 646
   - max_depth: 3
   - max_leaves: 98
   - min_child_weight: 4
   - reg_alpha: 0.7905569975811583
   - reg_lambda: 0.20348024132670334
   - min_frequency: 0.07218855115597657


*Se obtuvo un learning_rate moderado (0.077) y un alto número de árboles (n_estimators = 646), lo que permite un aprendizaje estable.* 

*La baja profundidad (max_depth = 3) junto a una alta cantidad de hojas (max_leaves = 98) sugiere árboles poco profundos.*

*El valor elevado de min_child_weight (4) y la fuerte regularización L1 (reg_alpha = 0.79) indican un modelo que evita sobreajuste. La regularización L2 (reg_lambda = 0.20) complementa el control de complejidad del modelo.*

*El parámetro min_frequency = 0.072 en el OneHotEncoder filtra categorías poco frecuentes, mejorando la generalización.*

*Los hiperparámetros hacen sentido y reflejan un modelo robusto y regularizado.*

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
#!pip install optuna-integration[xgboost]

In [ ]:
# Inserte su código acá

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# Inserte su código acá

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# Inserte su código acá

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>